Run the below four cells to obtain all libraries and mount drive if your dataset is stored in google drive.

#Import Models and libraries

In [1]:
!nvidia-smi

Fri Oct  2 02:46:48 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.23.05    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
cd drive/My\ Drive/Text\ Graphs\ 2020\ Final\ Folder/

/content/drive/.shortcut-targets-by-id/1_wfQNsSYHAtbbrf6AXTLdA8JaFzRM3KF/Text Graphs 2020 Final Folder


In [4]:
!pip install transformers
import nltk
nltk.download('stopwords')
from pathlib import Path
from tqdm.notebook import tqdm
from tqdm import trange
import pandas as po
import numpy as np
import warnings
import pickle
import nltk
import math
from paths import get_path_q, get_path_df_scores, get_path_predict
from utils import get_df_explanations, get_questions, obtain_model_names_and_classes, obtain_useful
from rank import get_ranks, get_preds, ideal_rerank, remove_combo_suffix, format_predict_line
from train_and_eval_functions import train_model, evaluate_model, predict_model
SEP = "#" * 100 + "\n"
path_data   = Path("data/")
path_tables = path_data.joinpath("raw/tables")
device = 'cuda'

     |████████████████████████████████| 1.1MB 4.4MB/s 
     |████████████████████████████████| 1.1MB 12.5MB/s 
     |████████████████████████████████| 890kB 23.3MB/s 
     |████████████████████████████████| 3.0MB 27.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=57b335980c913eba642a03564b23271c886a3986c82f5070bbe7d1e75f1a05cf
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


#Train Model

Run the following set of cells to train the model. Specify the model name and model type from the list of options provided to you. Make sure the specific set of models are available of training.


####This will create new intermidiate variables to be used


In [ ]:
df, df_exp, uids, uid2idx, uid2text, ranks, preds  = obtain_useful(path_data, path_tables, mode='train',top_k = 100)

####This will save all the intermediate variables to be used

In [ ]:
with open('./data/df.pkl', 'wb') as f:
  pickle.dump(df, f)
with open('./data/df_exp.pkl', 'wb') as f:
  pickle.dump(df_exp, f)
with open('./data/uids.pkl', 'wb') as f:
  pickle.dump(uids, f)
with open('./data/uid2idx.pkl', 'wb') as f:
  pickle.dump(uid2idx, f)
with open('./data/uid2text.pkl', 'wb') as f:
  pickle.dump(uid2text, f)
with open('./data/ranks.pkl', 'wb') as f:
  pickle.dump(ranks, f)
with open('./data/preds.pkl', 'wb') as f:
  pickle.dump(preds, f)

####This will load all the intermediate variables to be used

In [5]:
with open('./data/df.pkl', 'rb') as f:
  df= pickle.load( f)
with open('./data/df_exp.pkl', 'rb') as f:
  df_exp= pickle.load(f)
with open('./data/uids.pkl', 'rb') as f:
  uids = pickle.load(f)
with open('./data/uid2idx.pkl', 'rb') as f:
  uid2idx = pickle.load(f)
with open('./data/uid2text.pkl', 'rb') as f:
  uid2text = pickle.load(f)
with open('./data/ranks.pkl', 'rb') as f:
  ranks = pickle.load(f)
with open('./data/preds.pkl', 'rb') as f:
  preds = pickle.load(f)

## Run this to train the model

In [6]:
MODEL_CLASSES, model_with_no_token_types, tokenizer = obtain_model_names_and_classes(model_name='scibert', model_type='allenai/scibert_scivocab_uncased')

Choose from the list of models and their respective pretrained versions:
[{'bert': ['bert-base-uncased']}, {'xlm': ['xlm-mlm-en-2048']}, {'roberta': ['roberta-base']}, {'bart': ['facebook/bart-base']}, {'longformer': ['allenai/longformer-base-4096']}, {'albert': ['albert-xlarge-v2', 'albert-large-v2', 'albert-base-v2']}, {'electra': ['google/electra-large-generator']}, {'reformer': ['google/reformer-crime-and-punishment', 'google/reformer-enwik8']}, {'distilbert': ['distilbert-base-uncased']}, {'scibert': ['allenai/scibert_scivocab_uncased']}]


In [ ]:
save_model= train_model( df, df_exp, uids, uid2idx, uid2text, ranks, preds, MODEL_CLASSES,
                        model_with_no_token_types, model_name='scibert', model_type='allenai/scibert_scivocab_uncased', 
                        top_k = 300, num_train_epochs = 3, BATCH_SIZE=128, learning_rate=3e-5, epsilon=1e-8, 
                        gradient_accumulation_steps = 1, max_grad_norm = 1, weight_decay = 0, number_of_warmup_steps=0, 
                        global_step = 0, tr_loss=0.0, logging_loss = 0.0 )

Some weights of the model checkpoint at allenai/scibert_scivocab_uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification we

#Train MAP
You can run this cell after training the model or directly by loading the appropriate saved models and then evaluating the model to get the MAP score on Train DATASET.

####This will create new intermidiate variables to be used


In [ ]:
df, df_exp, uids, uid2idx, uid2text, ranks, preds  = obtain_useful(path_data, path_tables, mode='train',top_k = 100)

 15%|█▌        | 1368/9029 [00:00<00:00, 13670.85it/s]

Default lemmatizer (41883) concatenated (or not) with extras (162) -> 41883


100%|██████████| 9029/9029 [00:00<00:00, 14583.83it/s]


Explanation df shape: (9029, 5)


 41%|████▏     | 915/2207 [00:00<00:00, 9142.16it/s]

...

100%|██████████| 2207/2207 [00:00<00:00, 9085.66it/s]


(2207, 23)
Num canonical explanations: 9029
Num canonical explanations: 9029


100%|██████████| 2207/2207 [07:39<00:00,  4.80it/s]


Num canonical explanations: 9029


100%|██████████| 2207/2207 [00:00<00:00, 11715.10it/s]


####This will save all the intermediate variables to be used

In [ ]:
with open('./data/df.pkl', 'wb') as f:
  pickle.dump(df, f)
with open('./data/df_exp.pkl', 'wb') as f:
  pickle.dump(df_exp, f)
with open('./data/uids.pkl', 'wb') as f:
  pickle.dump(uids, f)
with open('./data/uid2idx.pkl', 'wb') as f:
  pickle.dump(uid2idx, f)
with open('./data/uid2text.pkl', 'wb') as f:
  pickle.dump(uid2text, f)
with open('./data/ranks.pkl', 'wb') as f:
  pickle.dump(ranks, f)
with open('./data/preds.pkl', 'wb') as f:
  pickle.dump(preds, f)

####This will load all the intermediate variables to be used

In [ ]:
with open('./data/df.pkl', 'rb') as f:
  df= pickle.load( f)
with open('./data/df_exp.pkl', 'rb') as f:
  df_exp= pickle.load(f)
with open('./data/uids.pkl', 'rb') as f:
  uids = pickle.load(f)
with open('./data/uid2idx.pkl', 'rb') as f:
  uid2idx = pickle.load(f)
with open('./data/uid2text.pkl', 'rb') as f:
  uid2text = pickle.load(f)
with open('./data/ranks.pkl', 'rb') as f:
  ranks = pickle.load(f)
with open('./data/preds.pkl', 'rb') as f:
  preds = pickle.load(f)

## Run this to get MAP on train

In [ ]:
## Evaluate Model on train dataset
MODEL_CLASSES, model_with_no_token_types, tokenizer = obtain_model_names_and_classes(model_name='bart', model_type='facebook/bart-base')
#save_model='albert_t_100_epoc_3_lr_2e-05_b_s_32'
evaluate_model(tokenizer,df, df_exp, uids, uid2idx, uid2text, ranks, preds, save_model, model_with_no_token_types, model_name='bart', mode='train', top_k = 300 )

Choose from the list of models and their respective pretrained versions:
[{'bert': ['bert-base-uncased']}, {'xlm': ['xlm-mlm-en-2048']}, {'roberta': ['roberta-base']}, {'bart': ['facebook/bart-base']}, {'longformer': ['allenai/longformer-base-4096']}, {'albert': ['albert-xlarge-v2', 'albert-large-v2', 'albert-base-v2']}, {'electra': ['google/electra-large-generator']}, {'reformer': ['google/reformer-crime-and-punishment', 'google/reformer-enwik8']}, {'distilbert': ['distilbert-base-uncased']}, {'scibert': ['allenai/scibert_scivocab_uncased']}]



0.45457135718794306
0.716655158902937


#Dev MAP
Running this cell below will give you the dev MAP score of the given Model  

####This will create new intermidiate variables to be used


In [ ]:
df, df_exp, uids, uid2idx, uid2text, ranks, preds  = obtain_useful(path_data, path_tables, mode='dev',top_k = 100)

 18%|█▊        | 1612/9029 [00:00<00:00, 16112.79it/s]

Default lemmatizer (41883) concatenated (or not) with extras (162) -> 41883


100%|██████████| 9029/9029 [00:00<00:00, 16516.20it/s]


Explanation df shape: (9029, 5)


100%|██████████| 496/496 [00:00<00:00, 9213.31it/s]

Default lemmatizer (41883) concatenated (or not) with extras (162) -> 41883
(496, 23)


Num canonical explanations: 9029
Num canonical explanations: 9029


100%|██████████| 496/496 [01:44<00:00,  4.75it/s]


Num canonical explanations: 9029


100%|██████████| 496/496 [00:00<00:00, 25093.48it/s]


####This will save all the intermediate variables to be used

In [ ]:
with open('./data/df_dev.pkl', 'wb') as f:
  pickle.dump(df, f)
with open('./data/df_exp_dev.pkl', 'wb') as f:
  pickle.dump(df_exp, f)
with open('./data/uids_dev.pkl', 'wb') as f:
  pickle.dump(uids, f)
with open('./data/uid2idx_dev.pkl', 'wb') as f:
  pickle.dump(uid2idx, f)
with open('./data/uid2text_dev.pkl', 'wb') as f:
  pickle.dump(uid2text, f)
with open('./data/ranks_dev.pkl', 'wb') as f:
  pickle.dump(ranks, f)
with open('./data/preds_dev.pkl', 'wb') as f:
  pickle.dump(preds, f)

####This will load all the intermediate variables to be used

In [ ]:
with open('./data/df_dev.pkl', 'rb') as f:
  df= pickle.load( f)
with open('./data/df_exp_dev.pkl', 'rb') as f:
  df_exp= pickle.load(f)
with open('./data/uids_dev.pkl', 'rb') as f:
  uids = pickle.load(f)
with open('./data/uid2idx_dev.pkl', 'rb') as f:
  uid2idx = pickle.load(f)
with open('./data/uid2text_dev.pkl', 'rb') as f:
  uid2text = pickle.load(f)
with open('./data/ranks_dev.pkl', 'rb') as f:
  ranks = pickle.load(f)
with open('./data/preds_dev.pkl', 'rb') as f:
  preds = pickle.load(f)

## Run this to get MAP on dev

In [ ]:
## Evaluate Model on train dataset
MODEL_CLASSES, model_with_no_token_types, tokenizer = obtain_model_names_and_classes(model_name='bart', model_type='facebook/bart-base')
#save_model='roberta_t_300_epoc_3_lr_2e-05_b_s_256'
evaluate_model(tokenizer,df, df_exp, uids, uid2idx, uid2text, ranks, preds, save_model, model_with_no_token_types, model_name='bart', mode='dev', top_k = 300 )

Choose from the list of models and their respective pretrained versions:
[{'bert': ['bert-base-uncased']}, {'xlm': ['xlm-mlm-en-2048']}, {'roberta': ['roberta-base']}, {'bart': ['facebook/bart-base']}, {'longformer': ['allenai/longformer-base-4096']}, {'albert': ['albert-xlarge-v2', 'albert-large-v2', 'albert-base-v2']}, {'electra': ['google/electra-large-generator']}, {'reformer': ['google/reformer-crime-and-punishment', 'google/reformer-enwik8']}, {'distilbert': ['distilbert-base-uncased']}, {'scibert': ['allenai/scibert_scivocab_uncased']}]



0.476474518279811
0.5036149218312401


#Test predictions
Running this cell below will give 

####This will create new intermidiate variables to be used


In [ ]:
df, df_exp, uids, uid2idx, uid2text, ranks, preds  = obtain_useful(path_data, path_tables, mode='test',top_k = 100)

 17%|█▋        | 1502/9029 [00:00<00:00, 15015.29it/s]

Default lemmatizer (41883) concatenated (or not) with extras (162) -> 41883


100%|██████████| 9029/9029 [00:00<00:00, 16288.98it/s]


Explanation df shape: (9029, 5)


 59%|█████▉    | 988/1664 [00:00<00:00, 9872.48it/s]

...

100%|██████████| 1664/1664 [00:00<00:00, 9508.74it/s]


(1664, 23)
Num canonical explanations: 9029
Num canonical explanations: 9029


100%|██████████| 1664/1664 [05:48<00:00,  4.78it/s]


Num canonical explanations: 9029


100%|██████████| 1664/1664 [00:00<00:00, 12427.43it/s]


####This will save all the intermediate variables to be used

In [ ]:
with open('./data/df_te.pkl', 'wb') as f:
  pickle.dump(df, f)
with open('./data/df_exp_te.pkl', 'wb') as f:
  pickle.dump(df_exp, f)
with open('./data/uids_te.pkl', 'wb') as f:
  pickle.dump(uids, f)
with open('./data/uid2idx_te.pkl', 'wb') as f:
  pickle.dump(uid2idx, f)
with open('./data/uid2text_te.pkl', 'wb') as f:
  pickle.dump(uid2text, f)
with open('./data/ranks_te.pkl', 'wb') as f:
  pickle.dump(ranks, f)
with open('./data/preds_te.pkl', 'wb') as f:
  pickle.dump(preds, f)

####This will load all the intermediate variables to be used

In [ ]:
with open('./data/df_te.pkl', 'rb') as f:
  df= pickle.load( f)
with open('./data/df_exp_te.pkl', 'rb') as f:
  df_exp= pickle.load(f)
with open('./data/uids_te.pkl', 'rb') as f:
  uids = pickle.load(f)
with open('./data/uid2idx_te.pkl', 'rb') as f:
  uid2idx = pickle.load(f)
with open('./data/uid2text_te.pkl', 'rb') as f:
  uid2text = pickle.load(f)
with open('./data/ranks_te.pkl', 'rb') as f:
  ranks = pickle.load(f)
with open('./data/preds_te.pkl', 'rb') as f:
  preds = pickle.load(f)

##Run this to get Predictions

In [ ]:
## Get Test predictions 
#save_model='roberta_top_100_epochs_32_batch.pt'
#save_model='roberta_t_500_epoc_3_lr_2e-05_b_s_256'
MODEL_CLASSES, model_with_no_token_types, tokenizer = obtain_model_names_and_classes(model_name='bart', model_type='facebook/bart-base')
predict_model(tokenizer,df, df_exp, uids, uid2idx, uid2text, ranks, preds, save_model, model_with_no_token_types, top_k=300, model_name='bart')

Choose from the list of models and their respective pretrained versions:
[{'bert': ['bert-base-uncased']}, {'xlm': ['xlm-mlm-en-2048']}, {'roberta': ['roberta-base']}, {'bart': ['facebook/bart-base']}, {'longformer': ['allenai/longformer-base-4096']}, {'albert': ['albert-xlarge-v2', 'albert-large-v2', 'albert-base-v2']}, {'electra': ['google/electra-large-generator']}, {'reformer': ['google/reformer-crime-and-punishment', 'google/reformer-enwik8']}, {'distilbert': ['distilbert-base-uncased']}, {'scibert': ['allenai/scibert_scivocab_uncased']}]



The predictions are stored in the file : ./predictions/bart_t_300_epoc_3_lr_2e-05_b_s_128.txt
